# Universidad del Valle de Guatemala
## Security Data Science - 10
* Juan Manuel Marroquin Alfaro - 19845
* Carlos Alberto Raxtum Ramos - 19721
* Jose Abraham Gutierrez Corado - 19111
* Walter Danilo Saldaña Salguero - 19897
* Javier Alejandro Cotto Argueta - 19324

## Proyecto Fase 2: Selección de caracteristicas y limpieza de datos

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np
import sklearn.cluster as cluster
from sklearn.cluster import KMeans
import itertools
from sklearn.preprocessing import scale
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from yellowbrick.cluster import SilhouetteVisualizer
from os import listdir
from os.path import isfile, join
import os

## Normal Images

In [2]:
df1 = pd.read_csv('normal_images_info.csv', header = None)
df1.sample(5)

,0,1,2,3,4,5,6,7,8,9,...,65527,65528,65529,65530,65531,65532,65533,65534,65535,65536
32,31.0,64,58,103,118,63,37,38,43,39,...,43,52,70,74,85,87,91,89,104,154
548,547.0,149,150,150,151,152,151,153,151,152,...,16,26,71,71,52,42,28,30,32,32
307,306.0,177,177,178,177,179,180,179,181,180,...,43,29,27,25,26,25,24,21,25,23
781,780.0,255,255,255,255,255,255,255,255,255,...,83,82,85,83,76,75,79,81,74,72
993,992.0,126,127,130,132,132,132,133,133,132,...,75,75,76,75,74,72,73,71,71,72


### Observaciones del Dataframe de Imagenes Normales

In [5]:
df1.shape

(1001, 65538)

## Stego Images

In [3]:
df2 = pd.read_csv('infected_images_info.csv', header = None)
df2.sample(5)

,0,1,2,3,4,5,6,7,8,9,...,65527,65528,65529,65530,65531,65532,65533,65534,65535,65536
790,789.0,72,63,56,49,31,8,16,50,47,...,45,48,48,46,45,42,42,30,37,57
452,451.0,111,143,136,51,60,90,75,73,78,...,195,205,200,193,154,128,123,121,116,108
810,809.0,189,188,186,184,183,183,183,184,185,...,15,12,13,16,12,12,14,14,13,12
379,378.0,111,74,53,31,26,25,28,31,37,...,113,114,113,110,109,110,110,112,111,111
123,122.0,72,41,21,53,53,56,39,27,9,...,117,109,116,114,116,108,112,111,114,106


### Observaciones del Dataframe de Imagenes Alteradas

In [6]:
df2.shape

(1001, 65538)

## Dataframe Completo

Concatenación de los dataframes

In [4]:

#1 Si tiene 0 No tiene
df1['isInfected'] = 0
df2['isInfected'] = 1

df  = pd.concat([df1, df2])
df

,0,1,2,3,4,5,6,7,8,9,...,65528,65529,65530,65531,65532,65533,65534,65535,65536,isInfected
0,NaN,0,1,2,3,4,5,6,7,8,...,65527,65528,65529,65530,65531,65532,65533,65534,65535,0
1,0.0,81,77,81,82,79,82,81,79,81,...,19,20,22,22,17,17,17,22,24,0
2,1.0,6,5,5,6,5,5,5,5,5,...,5,6,5,5,5,5,5,5,5,0
3,2.0,15,16,16,17,16,15,16,16,16,...,18,22,23,15,12,11,12,12,12,0
4,3.0,97,99,96,93,92,93,92,91,91,...,48,60,69,65,61,58,50,49,53,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,995.0,11,13,15,22,26,24,24,23,24,...,88,88,87,86,87,87,88,87,86,1
997,996.0,66,63,63,62,60,59,49,65,76,...,180,178,187,169,150,137,165,148,85,1
998,997.0,86,87,89,91,84,91,91,88,91,...,58,49,51,58,44,44,45,39,44,1
999,998.0,129,128,150,174,159,162,157,144,130,...,103,102,103,100,88,84,84,76,27,1


### Observaciones del Dataframe completo

In [8]:
df.shape

(2002, 65538)